In [4]:
import json
import datetime


def getDayPosition(json_profile):
    with open(json_profile) as json_file:
        data = json.load(json_file)
        # hurricane_days['ID'] = []
        # hurricane_days['ID'].append(data['id'])

    date_set = set() # set to avoid duplicates
    long_lat = []
    for path in data['path']:
        str_date = str(path['date'])
        # separate date into year month day to add hypens in between
        year = str_date[0:4]
        month = str_date[4:6]
        day = str_date[6:8]
        new_date = year + '-' + month + '-' + day
        date_set.add(new_date)

        # trying to add 1 to datetime -- wrong format?
        #start_day = datetime.datetime.strptime(new_date, '%y-%m-%d')
        #end_day = start_day + datetime.timedelta(days=1)

        # trying to create dictionary so that when I add a new set of long/lat coords to a date, 
            # the function will check if there is already a set of coords
            # if there is, it will move to the next date and coord pair
        # hurricane_days['ID'][0] = []
        # hurricane_days['ID'][0].append(new_date)

        long = path['lg'][0:4]
        lat = path['lt'][0:4]
        long_lat.append([long, lat])

    return date_set, long_lat


In [3]:
#getDayPosition('/home/fun/data/AL182012/profile.json')

In [5]:
#create a function that takes HDF5 lists  and output into nparray (csv file)
#'/home/fun/hurricane_intensity/3B-HHR.MS.MRG.3IMERG.20121021-S170000-E172959.1020.V06B.HDF5'

def post_processing(param, folder, full_path_filename, lonstr, latstr): # lon and lat should be read from the path name or given externally
    '''
        1) pick the data only in bounding  box
        2) output to CSV with date_time
        3) remove the downloaded hdf5 files
    '''    
    lon = 0
    lat = 0
    sign = -1.0 if 'W' in lonstr else 1.0
    lon = sign * float(lonstr[:-1])
    sign = -1.0 if 'S' in latstr else 1.0
    lat = sign * float(latstr[:-1])
            
    x = int((lon-longitude[0])*10)
    y = int((lat-latitude[0])*10)
    
    hdf_array = h5py.File(full_path_filename, 'r')
    data = hdf_array['Grid']['precipitationCal'][:, x-5:x+5, y-5:y+5] # 10 by 10, one square degree (#1)

    datestr = str(param['date'])
    timestr = str(param['time'])
    final = np.array([data])

    pathname = folder + 'imerg_precipitation_' + datestr + '_' + timestr + '.npy'
    np.save(pathname, final) # write np array to .npy file with pre-existing file name + datestr + timestr (#2)

    os.remove(full_path_filename) # remove previous .HDF5 file (#3)